In [ ]:
from modulefinder import packagePathMap
from unittest import result
from click import pass_obj
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction.text import TfidfVectorizer
from simcse import SimCSE
import json
import jsonlines
import numpy as np

In [ ]:
model_sent_bert_nli = SentenceTransformer('nli-roberta-base-v2')

03/15/2022 16:51:15 - INFO - sentence_transformers.SentenceTransformer -   Load pretrained SentenceTransformer: nli-roberta-base-v2
03/15/2022 16:51:46 - INFO - sentence_transformers.SentenceTransformer -   Use pytorch device: cuda


In [ ]:
model_sent_bert_pp = SentenceTransformer('paraphrase-TinyBERT-L6-v2')

03/15/2022 16:58:43 - INFO - sentence_transformers.SentenceTransformer -   Load pretrained SentenceTransformer: paraphrase-TinyBERT-L6-v2
03/15/2022 16:59:01 - INFO - sentence_transformers.SentenceTransformer -   Use pytorch device: cuda


In [ ]:
model_unsimcse = SimCSE("princeton-nlp/unsup-simcse-bert-base-uncased")

03/15/2022 17:17:49 - INFO - simcse.tool -   Use `cls_before_pooler` for unsupervised models. If you want to use other pooling policy, specify `pooler` argument.


In [ ]:
model_susimcse = SimCSE("princeton-nlp/sup-simcse-bert-base-uncased")

In [ ]:
tokenizer_specter = AutoTokenizer.from_pretrained('allenai/specter')

In [ ]:
model_specter = AutoModel.from_pretrained('allenai/specter')

In [ ]:
tokenizer_scibert_uncased = AutoTokenizer.from_pretrained(
    'allenai/scibert_scivocab_uncased')

In [ ]:
model_scibert_uncased = AutoModel.from_pretrained(
    'allenai/scibert_scivocab_uncased')

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenizer_scibert_cased = AutoTokenizer.from_pretrained(
    'allenai/scibert_scivocab_cased')

In [ ]:
model_scibert_cased = AutoModel.from_pretrained(
    'allenai/scibert_scivocab_cased')

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def get_bert_nli_embedding(sentence):
    return model_sent_bert_nli.encode(sentence)


def get_bert_pp_embedding(sentence):
    return model_sent_bert_pp.encode(sentence)


def get_specter_embedding(sentence):
    # TODO: check if max_length is required
    inputs = tokenizer_specter(
        sentence, padding=True, truncation=True, return_tensors="pt", max_length=65536)
    return model_specter(**inputs).last_hidden_state[:, 0, :]


def get_scibert_uncased_embedding(sentence):
    inputs = tokenizer_scibert_uncased(
        sentence, padding=True, truncation=True, return_tensors="pt", max_length=65536)
    return model_scibert_uncased(**inputs).last_hidden_state[:, 0, :]


def get_scibert_cased_embedding(sentence):
    inputs = tokenizer_scibert_cased(
        sentence, padding=True, truncation=True, return_tensors="pt", max_length=65536)
    return model_scibert_cased(**inputs).last_hidden_state[:, 0, :]


def get_unsimcse_embedding(sentence):
    return model_unsimcse.encode(sentence)


def get_susimcse_embedding(sentence):
    return model_susimcse.encode(sentence)

def tfidf_fit_corpus(corpus):
    tfidf = TfidfVectorizer().fit([corpus])
    return tfidf

def get_tfidf_embedding(tfidf, query):
    return tfidf.transform([query]).toarray()[0]


def get_bm25_score(corpus, query):
    tokenized_corpus = [doc.split(" ") for doc in corpus]
    tokenized_query = query.split(" ")
    bm25 = BM25Okapi(tokenized_corpus)
    return bm25.get_scores(tokenized_query)

In [ ]:

class Paper:
    def __init__(self, paper_id, metadata, title, abstract, pred_labels_truncated, pred_labels):
        self.paper_id = paper_id
        self.metadata = metadata
        self.title = title
        self.abstract = abstract
        self.pred_labels_truncated = pred_labels_truncated
        self.pred_labels = pred_labels

papers = dict()
with jsonlines.open('../data/abstracts-csfcube-preds.jsonl') as reader:
    for obj in reader:
        papers[obj['paper_id']] = Paper(obj['paper_id'], obj['metadata'], obj['title'], obj['abstract'], obj['pred_labels_truncated'], obj['pred_labels'])

f = open('../data/test-pid2anns-csfcube-background.json')
q_background = json.load(f)
q_background_ids = list(q_background.keys())

f = open('../data/test-pid2anns-csfcube-method.json')
q_method = json.load(f)
q_method_ids = list(q_method.keys())

f = open('../data/test-pid2anns-csfcube-result.json')
q_result = json.load(f)
q_result_ids = list(q_result.keys())

all_ids = list(papers.keys())

# candidate_ids = list(set(all_ids) - set(q_background_ids) - set(q_method_ids) - set(q_result_ids))

print(len(all_ids))
print(len(q_background_ids))
print(len(q_method_ids))
print(len(q_result_ids))
# print(len(candidate_ids))

def facet_sentences(pid, facet):
    labels = {'background': ['background_label', 'objective_label'], 'method': ['method_label'], 'result': ['result_label']}
    paper = papers[pid]
    s = list()
    for i in range(len(paper.pred_labels)):
        if paper.pred_labels[i] in labels[facet]:
            s.append(paper.abstract[i])
    return s

def merge_list_to_str(l):
    return " ".join(l)



4207
16
17
17


In [ ]:
corpus_list = list()
for pid in papers:
    doc = papers[pid].abstract
    corpus_list = corpus_list + doc
corpus = merge_list_to_str(corpus_list)

In [ ]:
tfidf = tfidf_fit_corpus(corpus)

In [ ]:
# tf_idf all
d = dict()
err_pids = list()
i = 0
for pid in papers:
    i += 1
    if i % 50 == 0:
        print(i)
    try:
        doc = papers[pid].abstract
        if len(doc) == 0:
            d[pid] = list()
            continue
        bg_tfidf_array = np.array(get_tfidf_embedding(tfidf, merge_list_to_str(doc))).tolist()
        d[pid] = bg_tfidf_array
    except:
        err_pids.append(pid)
print(f'len(err_pids) = {len(err_pids)}')
print(err_pids)
with open("embeddings/tf_idf/all.json", "w") as outfile:
    json.dump(d, outfile)
d = json.load(open('embeddings/tf_idf/all.json'),
              parse_float=lambda x: round(float(x), 6))
with open('embeddings/tf_idf/all.json', "w") as outfile:
    json.dump(d, outfile)


50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
len(err_pids) = 0
[]


In [ ]:
# tf_idf queries
for i, ids in enumerate([q_background_ids, q_method_ids, q_result_ids]):
    d = dict()
    err_pids = list()
    s = ''
    if i == 0:
        s = 'background'
    elif i == 1:
        s = 'method'
    elif i == 2:
        s = 'result'
    for pid in ids:
        try:
            doc = facet_sentences(pid, s)
            if len(doc) == 0:
                d[pid] = list()
                continue
            tfidf_array = np.array(get_tfidf_embedding(tfidf, merge_list_to_str(doc))).tolist()
            d[pid] = tfidf_array
        except:
            err_pids.append(pid)
    print(f'len(err_pids), {s} = {len(err_pids)}')
    print(err_pids)
    with open(f'embeddings/tf_idf/{s}.json', "w") as outfile:
        json.dump(d, outfile)
    d = json.load(open(f'embeddings/tf_idf/{s}.json'),
                parse_float=lambda x: round(float(x), 6))
    with open(f'embeddings/tf_idf/{s}.json', "w") as outfile:
        json.dump(d, outfile)


len(err_pids), background = 0
[]
len(err_pids), method = 0
[]
len(err_pids), result = 0
[]


In [ ]:
# bert_nli all
d = dict()
err_pids = list()
i = 0
for pid in papers:
    # i += 1
    # if i % 50 == 0:
    #     print(i)
    try:
        doc = papers[pid].abstract
        if len(doc) == 0:
            d[pid] = list()
            continue
        bert_nli_array = np.array(get_bert_nli_embedding(merge_list_to_str(doc))).tolist()
        d[pid] = bert_nli_array
    except:
        err_pids.append(pid)
print(f'len(err_pids) = {len(err_pids)}')
print(err_pids)
with open("embeddings/bert_nli/all.json", "w") as outfile:
    json.dump(d, outfile)
d = json.load(open('embeddings/bert_nli/all.json'),
              parse_float=lambda x: round(float(x), 6))
with open('embeddings/bert_nli/all.json', "w") as outfile:
    json.dump(d, outfile)


In [ ]:
# bert_nli queries
for i, ids in enumerate([q_background_ids, q_method_ids, q_result_ids]):
    d = dict()
    err_pids = list()
    s = ''
    if i == 0:
        s = 'background'
    elif i == 1:
        s = 'method'
    elif i == 2:
        s = 'result'
    for pid in ids:
        try:
            doc = facet_sentences(pid, s)
            if len(doc) == 0:
                d[pid] = list()
                continue
            bert_nli_array = np.array(get_bert_nli_embedding(merge_list_to_str(doc))).tolist()
            d[pid] = bert_nli_array
        except:
            err_pids.append(pid)
    print(f'len(err_pids), {s} = {len(err_pids)}')
    print(err_pids)
    with open(f'embeddings/bert_nli/{s}.json', "w") as outfile:
        json.dump(d, outfile)
    d = json.load(open(f'embeddings/bert_nli/{s}.json'),
                parse_float=lambda x: round(float(x), 6))
    with open(f'embeddings/bert_nli/{s}.json', "w") as outfile:
        json.dump(d, outfile)


In [ ]:
# bert_pp all
d = dict()
err_pids = list()
i = 0
for pid in papers:
    # i += 1
    # if i % 50 == 0:
    #     print(i)
    try:
        doc = papers[pid].abstract
        if len(doc) == 0:
            d[pid] = list()
            continue
        bert_pp_array = np.array(get_bert_pp_embedding(merge_list_to_str(doc))).tolist()
        d[pid] = bert_pp_array
    except:
        err_pids.append(pid)
print(f'len(err_pids) = {len(err_pids)}')
print(err_pids)
with open("embeddings/bert_pp/all.json", "w") as outfile:
    json.dump(d, outfile)
d = json.load(open('embeddings/bert_pp/all.json'),
              parse_float=lambda x: round(float(x), 6))
with open('embeddings/bert_pp/all.json', "w") as outfile:
    json.dump(d, outfile)

In [ ]:
# bert_pp queries
for i, ids in enumerate([q_background_ids, q_method_ids, q_result_ids]):
    d = dict()
    err_pids = list()
    s = ''
    if i == 0:
        s = 'background'
    elif i == 1:
        s = 'method'
    elif i == 2:
        s = 'result'
    for pid in ids:
        try:
            doc = facet_sentences(pid, s)
            if len(doc) == 0:
                d[pid] = list()
                continue
            bert_pp_array = np.array(get_bert_pp_embedding(merge_list_to_str(doc))).tolist()
            d[pid] = bert_pp_array
        except:
            err_pids.append(pid)
    print(f'len(err_pids), {s} = {len(err_pids)}')
    print(err_pids)
    with open(f'embeddings/bert_pp/{s}.json', "w") as outfile:
        json.dump(d, outfile)
    d = json.load(open(f'embeddings/bert_pp/{s}.json'),
                parse_float=lambda x: round(float(x), 6))
    with open(f'embeddings/bert_pp/{s}.json', "w") as outfile:
        json.dump(d, outfile)

In [ ]:
# unsimcse all
d = dict()
err_pids = list()
i = 0
for pid in papers:
    # if i == 1:
    #     break
    # i += 1
    # if i % 50 == 0:
    #     print(i)
    try:
        doc = papers[pid].abstract
        if len(doc) == 0:
            d[pid] = list()
            continue
        unsimcse_array = get_unsimcse_embedding(merge_list_to_str(doc)).detach().numpy().tolist()
        # print(len(unsimcse_array))
        d[pid] = unsimcse_array
    except:
        err_pids.append(pid)
print(f'len(err_pids) = {len(err_pids)}')
print(err_pids)
with open("embeddings/unsimcse/all.json", "w") as outfile:
    json.dump(d, outfile)
d = json.load(open('embeddings/unsimcse/all.json'),
              parse_float=lambda x: round(float(x), 6))
with open('embeddings/unsimcse/all.json', "w") as outfile:
    json.dump(d, outfile)

In [ ]:
# unsimcse queries
for i, ids in enumerate([q_background_ids, q_method_ids, q_result_ids]):
    d = dict()
    err_pids = list()
    s = ''
    if i == 0:
        s = 'background'
    elif i == 1:
        s = 'method'
    elif i == 2:
        s = 'result'
    for pid in ids:
        try:
            doc = facet_sentences(pid, s)
            if len(doc) == 0:
                d[pid] = list()
                continue
            unsimcse_array = get_unsimcse_embedding(merge_list_to_str(doc)).detach().numpy().tolist()
            d[pid] = unsimcse_array
        except:
            err_pids.append(pid)
    print(f'len(err_pids), {s} = {len(err_pids)}')
    print(err_pids)
    with open(f'embeddings/unsimcse/{s}.json', "w") as outfile:
        json.dump(d, outfile)
    d = json.load(open(f'embeddings/unsimcse/{s}.json'),
                parse_float=lambda x: round(float(x), 6))
    with open(f'embeddings/unsimcse/{s}.json', "w") as outfile:
        json.dump(d, outfile)

In [ ]:
# susimcse all
d = dict()
err_pids = list()
i = 0
for pid in papers:
    # i += 1
    # if i % 50 == 0:
    #     print(i)
    try:
        doc = papers[pid].abstract
        if len(doc) == 0:
            d[pid] = list()
            continue
        susimcse_array = get_susimcse_embedding(merge_list_to_str(doc)).detach().numpy().tolist()
        d[pid] = susimcse_array
    except:
        err_pids.append(pid)
print(f'len(err_pids) = {len(err_pids)}')
print(err_pids)
with open("embeddings/susimcse/all.json", "w") as outfile:
    json.dump(d, outfile)
d = json.load(open('embeddings/susimcse/all.json'),
              parse_float=lambda x: round(float(x), 6))
with open('embeddings/susimcse/all.json', "w") as outfile:
    json.dump(d, outfile)

In [ ]:
# susimcse queries
for i, ids in enumerate([q_background_ids, q_method_ids, q_result_ids]):
    d = dict()
    err_pids = list()
    s = ''
    if i == 0:
        s = 'background'
    elif i == 1:
        s = 'method'
    elif i == 2:
        s = 'result'
    for pid in ids:
        try:
            doc = facet_sentences(pid, s)
            if len(doc) == 0:
                d[pid] = list()
                continue
            susimcse_array = get_susimcse_embedding(merge_list_to_str(doc)).detach().numpy().tolist()
            d[pid] = susimcse_array
        except:
            err_pids.append(pid)
    print(f'len(err_pids), {s} = {len(err_pids)}')
    print(err_pids)
    with open(f'embeddings/susimcse/{s}.json', "w") as outfile:
        json.dump(d, outfile)
    d = json.load(open(f'embeddings/susimcse/{s}.json'),
                parse_float=lambda x: round(float(x), 6))
    with open(f'embeddings/susimcse/{s}.json', "w") as outfile:
        json.dump(d, outfile)

In [ ]:
# specter all
d = dict()
err_pids = list()
i = 0
for pid in papers:
    # if i == 1:
    #     break
    # i += 1
    # if i % 50 == 0:
    #     print(i)
    try:
        doc = papers[pid].abstract
        if len(doc) == 0:
            d[pid] = list()
            continue
        specter_array = get_specter_embedding(merge_list_to_str(doc)).detach().numpy().tolist()
        print(len(specter_array[0]))
        # print(specter_array)
        d[pid] = specter_array[0]
    except:
        err_pids.append(pid)
print(f'len(err_pids) = {len(err_pids)}')
print(err_pids)
with open("embeddings/specter/all.json", "w") as outfile:
    json.dump(d, outfile)
d = json.load(open('embeddings/specter/all.json'),
              parse_float=lambda x: round(float(x), 6))
with open('embeddings/specter/all.json', "w") as outfile:
    json.dump(d, outfile)

768
len(err_pids) = 0
[]


In [ ]:
# specter queries
for i, ids in enumerate([q_background_ids, q_method_ids, q_result_ids]):
    d = dict()
    err_pids = list()
    s = ''
    if i == 0:
        s = 'background'
    elif i == 1:
        s = 'method'
    elif i == 2:
        s = 'result'
    for pid in ids:
        try:
            doc = facet_sentences(pid, s)
            if len(doc) == 0:
                d[pid] = list()
                continue
            specter_array = get_specter_embedding(merge_list_to_str(doc)).detach().numpy().tolist()
            d[pid] = specter_array[0]
        except:
            err_pids.append(pid)
    print(f'len(err_pids), {s} = {len(err_pids)}')
    print(err_pids)
    with open(f'embeddings/specter/{s}.json', "w") as outfile:
        json.dump(d, outfile)
    d = json.load(open(f'embeddings/specter/{s}.json'),
                parse_float=lambda x: round(float(x), 6))
    with open(f'embeddings/specter/{s}.json', "w") as outfile:
        json.dump(d, outfile)

In [ ]:
# scibert uncased all
d = dict()
err_pids = list()
i = 0
for pid in papers:
    # if i == 1:
    #     break
    i += 1
    if i % 50 == 0:
        print(i)
    try:
        doc = papers[pid].abstract
        if len(doc) == 0:
            d[pid] = list()
            continue
        scibert_uncased_array = get_scibert_uncased_embedding(merge_list_to_str(doc)).detach().numpy().tolist()
        # print(len(scibert_uncased_array[0]))
        # print(scibert_uncased_array)
        d[pid] = scibert_uncased_array[0]
    except:
        err_pids.append(pid)
print(f'len(err_pids) = {len(err_pids)}')
print(err_pids)
with open("embeddings/scibert_uncased/all.json", "w") as outfile:
    json.dump(d, outfile)
d = json.load(open('embeddings/scibert_uncased/all.json'),
              parse_float=lambda x: round(float(x), 6))
with open('embeddings/scibert_uncased/all.json', "w") as outfile:
    json.dump(d, outfile)

768
len(err_pids) = 0
[]


In [ ]:
# scibert_uncased queries
for i, ids in enumerate([q_background_ids, q_method_ids, q_result_ids]):
    d = dict()
    err_pids = list()
    s = ''
    if i == 0:
        s = 'background'
    elif i == 1:
        s = 'method'
    elif i == 2:
        s = 'result'
    for pid in ids:
        try:
            doc = facet_sentences(pid, s)
            if len(doc) == 0:
                d[pid] = list()
                continue
            scibert_uncased_array = get_scibert_uncased_embedding(merge_list_to_str(doc)).detach().numpy().tolist()
            d[pid] = scibert_uncased_array[0]
        except:
            err_pids.append(pid)
    print(f'len(err_pids), {s} = {len(err_pids)}')
    print(err_pids)
    with open(f'embeddings/scibert_uncased/{s}.json', "w") as outfile:
        json.dump(d, outfile)
    d = json.load(open(f'embeddings/scibert_uncased/{s}.json'),
                parse_float=lambda x: round(float(x), 6))
    with open(f'embeddings/scibert_uncased/{s}.json', "w") as outfile:
        json.dump(d, outfile)

In [ ]:
# scibert cased all
d = dict()
err_pids = list()
i = 0
for pid in papers:
    # if i == 1:
    #     break
    i += 1
    if i % 50 == 0:
        print(i)
    try:
        doc = papers[pid].abstract
        if len(doc) == 0:
            d[pid] = list()
            continue
        scibert_cased_array = get_scibert_cased_embedding(merge_list_to_str(doc)).detach().numpy().tolist()
        # print(len(scibert_cased_array[0]))
        # print(scibert_cased_array)
        d[pid] = scibert_cased_array[0]
    except:
        err_pids.append(pid)
print(f'len(err_pids) = {len(err_pids)}')
print(err_pids)
with open("embeddings/scibert_cased/all.json", "w") as outfile:
    json.dump(d, outfile)
d = json.load(open('embeddings/scibert_cased/all.json'),
              parse_float=lambda x: round(float(x), 6))
with open('embeddings/scibert_cased/all.json', "w") as outfile:
    json.dump(d, outfile)

768
len(err_pids) = 0
[]


In [ ]:
# scibert_cased queries
for i, ids in enumerate([q_background_ids, q_method_ids, q_result_ids]):
    d = dict()
    err_pids = list()
    s = ''
    if i == 0:
        s = 'background'
    elif i == 1:
        s = 'method'
    elif i == 2:
        s = 'result'
    for pid in ids:
        try:
            doc = facet_sentences(pid, s)
            if len(doc) == 0:
                d[pid] = list()
                continue
            scibert_cased_array = get_scibert_cased_embedding(merge_list_to_str(doc)).detach().numpy().tolist()
            d[pid] = scibert_cased_array[0]
        except:
            err_pids.append(pid)
    print(f'len(err_pids), {s} = {len(err_pids)}')
    print(err_pids)
    with open(f'embeddings/scibert_cased/{s}.json', "w") as outfile:
        json.dump(d, outfile)
    d = json.load(open(f'embeddings/scibert_cased/{s}.json'),
                parse_float=lambda x: round(float(x), 6))
    with open(f'embeddings/scibert_cased/{s}.json', "w") as outfile:
        json.dump(d, outfile)